## POC - Transportation Sector
### City: Boston

This notebook contains some calculations for GHG emissions for the transportation sector in Boston city in 2019, based on the GPC protocol.

In [1]:
import pandas as pd

### GPC Formulas

Modern fuels are often blends of (bio)ethanol or biodiesel with fossil fuel derived diesel or gasoline

In [2]:
def EF_biogenicC02():
    """
    CO2 emissions resulting from combustion of ethanol or biodiesel
    
    Source
    ------
    see Ch.7 of “GHG Protocol for Cities”
    https://ghgprotocol.org/ghg-protocol-cities
    section 7.3.2
    """
    
    EF_biogenicCO2 = EF_ethanol*percentage_ethanol
    return EF_biogenicCO2

In [3]:
def EF_NonBiogenicCO2():
    """
    CO2 emissions resulting from combustion of fossil-fuel derived element
    Source
    ------
    see Ch.7 of “GHG Protocol for Cities”
    https://ghgprotocol.org/ghg-protocol-cities
    section 7.3.2
    """
    
    EF_nonbiogenicCO2 = EF_co2_petroleum*(100-percentage_ethanol)
    return EF_nonbiogenicCO2

In [4]:
def EF_CH4():
    """
    
    Source
    ------
    see Ch.7 of “GHG Protocol for Cities”
    https://ghgprotocol.org/ghg-protocol-cities
    section 7.3.2
    """
    
    EF_ch4 = EF_ch4_ethanol*percentage_ethanol + EF_ch4_petroleum*(100-percentage_ethanol)
    return EF_ch4

In [5]:
def EF_NO2():
    """
    
    Source
    ------
    see Ch.7 of “GHG Protocol for Cities”
    https://ghgprotocol.org/ghg-protocol-cities
    section 7.3.2
    """
    
    EF_no2 = EF_no2_ethanol*percentage_ethanol + EF_no2_petroleum*(100-percentage_ethanol)
    return EF_no2

# Data Analysis

## Fuel sales method

The idea here is:
- Find the total number of fuel stations in Boston
- Find information about total fuel sales in a city level, ideally categorized by fuel type sales

if not:
- Find information about total fuel sales in a state level or country level, ideally categorized by fuel type sales and scale it down

### General Information

In [6]:
#Population 2019

pop_US = 328300000
pop_Ma = 68950000 
pop_Bos = 694661

In [7]:
#Emission Factors

#Mobile Combustion CO2 Emission Factor
CO2_EF_gasoline = 8.78 #kg CO2 per Gallon
CO2_EF_diesel = 10.21 #kg CO2 per Gallon
CO2_EF_NG = 0.0545 #kg CO2 per scf

#Light-rail systems 
#reference = https://www.apta.com/wp-content/uploads/Standards_Documents/APTA-SUDS-CC-RP-001-09_Rev-1.pdf
EF_Light_Rail = 0.365 #National Average Weighted  by PMT (lbs CO2/PMT)
EF_Heavy_Rail = 0.224  #National Average Weighted  by PMT (lbs CO2/PMT)

#Not included to avoid double account
EF_Bus = 0.643 #National Average Weighted  by PMT (lbs CO2/PMT)

EF_others = 4.34 # mean of Range1 (lbs CO2/PMT) 

#https://www.eia.gov/environment/emissions/co2_vol_mass.php
EF_Distillate_Oil = 10.19 # Kilograms CO2 / gallon 
EF_Distillate_Oil = 10.19*0.00110231 # Tonnes CO2 / gallon 

EF_Resodual_Oil = 11.27 # Kilograms CO2 / gallon 
EF_Resodual_Oil = 11.27*0.00110231 # Tonnes CO2 / gallon 

#source: https://theicct.org/publication/co2-emissions-from-commercial-aviation-2013-2018-and-2019/#:~:text=On%20average%2C%20passenger%20aviation%20emitted,per%20year%20for%20international%20flights.
ef_flights = 90 #grams of CO2 per passenger-kilometer in 2019
EF_flights = 90*(1/0.621371)*(1/1000000) #Tonnes of CO2 / miles

### On-road Transportation estimation

In [8]:
#Source: https://www.fhwa.dot.gov/policyinformation/motorfuel/dec19/dec19.pdf
gasoline_2019_MA = 2803731456 #gallons

In [9]:
#Source: https://www.statista.com/statistics/1320482/monthly-us-diesel-consumption/
diesel_2019_US_barrels = 48220000 #barrels per day
diesel_2019_US_barrels = 48220000*365 #barrels in 2019

#If using U.S. oil barrels
#Number of barrels * 42 gallons/barrel = Equivalent volume in gallons

diesel_2019_US = diesel_2019_US_barrels*42 #gallons

In [10]:
#Natural Gas
#Natural Gas Consumption by End Use
#Transportation - Vehicle Fuel
#https://www.eia.gov/dnav/ng/ng_cons_sum_dcu_SMA_a.htm

Natural_Gas_MA = 130 #Million Cubic Feet MCF
Natural_Gas_MA = 130000000 #standard cubic feet (SCF)

In [11]:
gasoline_Bos = (gasoline_2019_MA*pop_Bos)/(pop_Ma) #gallons
diesel_Bos = (diesel_2019_US*pop_Bos)/(pop_US) #gallons
NG_Bos = (Natural_Gas_MA*pop_Bos)/(pop_US) #SCF

print(gasoline_Bos)
print(diesel_Bos)
print(NG_Bos)

28247177.62083272
1564124775.901919
275071.3676515382


In [12]:
emissions_CO2_Gasoline = (gasoline_Bos * CO2_EF_gasoline)*(0.00110231) #Tonnes of CO2
emissions_CO2_Diesel = (diesel_Bos * CO2_EF_diesel)*(0.00110231) #Tonnes of CO2
emissions_CO2_NG = (NG_Bos * CO2_EF_NG)*(0.00110231) #Tonnes of CO2

print(emissions_CO2_Gasoline)
print(emissions_CO2_Diesel)
print(emissions_CO2_NG)

273384.1450690726
17603575.397406578
16.52515860054021


### Railways

In [13]:
#MBTA
#reference -> https://www.apta.com/wp-content/uploads/2022-Q4-Ridership-APTA.pdf
Light_Rail = 30705000 #passenger miles travelled (PMT)
Heavy_Rail = 80969900 #passenger miles travelled (PMT)

Bus = 4603307 #passenger miles travelled (PMT)

In [14]:
Emissions_Light_Rail = Light_Rail * EF_Light_Rail * 0.0005 #Tonnes of CO2
Emissions_Heavy_Rail = Heavy_Rail * EF_Heavy_Rail * 0.0005 #Tonnes of CO2

print(Emissions_Light_Rail)
print(Emissions_Heavy_Rail)

5603.6625
9068.6288


### Waterborne navigation

In [15]:
#MBTA
#Source: https://www.mbta.com/performance-metrics/ridership-the-t
ferry = 69560 #passenger miles travelled (PMT)

In [16]:
Emissions_Ferry = ferry * EF_others * 0.0005 #Tonnes of CO2

print(Emissions_Ferry)

150.9452


### Aviation

In [17]:
#Source: https://www.transtats.bts.gov/Data_Elements.aspx?Data=1
#Airport Logan International

Domestic_flights = 19126938 #passenger
International_flights = 12795195 #passenger

In [ ]:
Emissions_Domestic_Flights = Domestic_flights * EF_flights
Emissions_International_Flights = International_flights * EF_flights

print(Emissions_Domestic_Flights)
print(Emissions_International_Flights)

### Electricity - Scope 2

In [18]:
#Adjusted Sales of Distillate Fuel Oil by End Use
#https://www.eia.gov/dnav/pet/pet_cons_821dsta_dcu_SMA_a.htm

Off_Highway_MA = 26789000 #gallons
Off_Highway_Bos = (Off_Highway_MA*pop_Bos)/(pop_Ma)
Emissions_Off_Highway = Off_Highway_Bos*EF_Distillate_Oil
print(Emissions_Off_Highway)

Railroad_MA = 19576000 #gallons
Railroad_Bos = (Railroad_MA*pop_Bos)/(pop_Ma)
Emissions_Railroad = Railroad_Bos*EF_Distillate_Oil
print(Emissions_Railroad)

On_Highway_MA = 438512000 #gallons
On_Highway_Bos = (On_Highway_MA*pop_Bos)/(pop_Ma)
Emissions_On_Highway = On_Highway_Bos*EF_Distillate_Oil
print(Emissions_On_Highway)

3031.6082496770523
2215.3407404411505
49624.71898101399


In [19]:
#Adjusted Sales of Distillate Fuel Oil by End Use
#https://www.eia.gov/dnav/pet/pet_cons_821dsta_dcu_SMA_a.htm

Vessel_Bunkering_MA_DFO = 9508000 #gallons
Vessel_Bunkering_DFO = (Vessel_Bunkering_MA_DFO*pop_Bos)/(pop_Ma)
Emissions_Vessel_Bunkering_DFO = Vessel_Bunkering_DFO*EF_Distillate_Oil
print(Emissions_Vessel_Bunkering_DFO)

#Adjusted Sales of Residual Fuel Oil by End Use
#https://www.eia.gov/dnav/pet/pet_cons_821rsda_dcu_nus_a.htm

Vessel_Bunkering_MA_RFO = 8903000 #gallons
Vessel_Bunkering_RFO = (Vessel_Bunkering_MA_RFO*pop_Bos)/(pop_Ma)
Emissions_Vessel_Bunkering_RFO = Vessel_Bunkering_RFO*EF_Resodual_Oil
print(Emissions_Vessel_Bunkering_RFO)

1075.9838455309798
1114.301418526656
